### Train model in notebook (inefficient, for testing and debug only)

In [1]:
import os

In [ ]:
#os.environ['NCCL_DEBUG'] = 'INFO'
#os.environ['NCCL_DEBUG_SUBSYS'] = 'ALL'
#os.environ['TORCH_DISTRIBUTED_DEBUG'] = 'INFO'
os.environ['NCCL_IB_DISABLE'] = '1'
os.environ['NCCL_SOCKET_IFNAME'] = 'enp2s0f1np1'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['NCCL_BLOCKING_WAIT'] = '1'

In [1]:
import sys
from os.path import join
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import optim, nn
import torch.multiprocessing as mp
from torch.distributed import init_process_group, destroy_process_group
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from astropy.io import fits
import pyxis.torch as pxt
import normflows as nf

from networks import *
from train import *
import config

train_dir = '/ocean/projects/phy250048p/shared/datasets/small'
test_dir = '/ocean/projects/phy250048p/shared/datasets/small'
fig_dir = '/ocean/projects/phy250048p/shared/figures/'
model_dir = '/ocean/projects/phy250048p/shared/models/'

In [2]:
torch.cuda.device_count()

1

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
world_size = 1
save_every = 1
nepochs = config.train['epoch_number']
batch_size = config.train['batch_size']
nfeatures = config.train['feature_number']

In [5]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12355"
torch.cuda.set_device(0)
init_process_group(backend='nccl', rank=0, world_size=1)

In [6]:
train_ds = pxt.TorchDataset(train_dir)
valid_ds = pxt.TorchDataset(test_dir)

In [7]:
model = ForkCNN(mode=1)
model.to(0)
model = DDP(model, device_ids=[0])

In [8]:
print(model)

DistributedDataParallel(
  (module): ForkCNN(
    (img_net): ImgCNN(
      (cnn_img): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (7): ResidualBlock(
          (cnn1): Sequential(
            (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (cnn2): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3),

In [8]:
# optimizer = optim.SGD(model.parameters(), 
#                       lr=config.train['initial_learning_rate'],
#                       momentum=config.train['momentum'])
optimizer = optim.AdamW(model.parameters(),
                        lr=config.train['initial_learning_rate'], 
                        weight_decay=config.train['weight_decay'])

In [9]:
trainer = CNNTrainer(world_size, model, nfeatures, train_ds, valid_ds, optimizer, 0, save_every, batch_size)

In [10]:
trainer.train(2)

/jet/home/xwang30/.conda/envs/kl-nn/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once


In [5]:
destroy_process_group()

AssertionError: 

In [5]:
mp.spawn(train_nn, args=(world_size, ForkCNN, CNNTrainer), nprocs=world_size)

INFO:Setup:Initializing
INFO:Setup:[rank: 0] Successfully set up device
INFO:Setup:Loaded model CNN-CNN-ffn_1m at epoch 79
INFO:Setup:[rank: 0] Successfully loaded training objects
INFO:Setup:[rank: 0] Successfully initialized Trainer
/jet/home/xwang30/.conda/envs/kl-nn/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once
INFO:Trainer:Setting up tensors on GPU
INFO:Trainer:Uploading training set to GPU...
INFO:Trainer:10% complete
INFO:Trainer:20% complete
INFO:Trainer:30% complete
INFO:Trainer:40% complete
INFO:Trainer:50% complete
INFO:Trainer:60% complete
INFO:Trainer:70% complete
INFO:Trainer:80% complete
INFO:Trainer:90% complete
INFO:Trainer:Uploading validation set to GPU...
INFO:Trainer:10% complete
INFO:Trainer:20% complete
INFO:Trainer:30% complete
INFO:Trainer:40% complete
INFO:Trainer

ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/jet/home/xwang30/.conda/envs/kl-nn/lib/python3.10/site-packages/torch/multiprocessing/spawn.py", line 95, in _wrap
    fn(i, *args)
  File "/jet/home/xwang30/kl-nn/arch/train.py", line 322, in train_nn
    trainer.train(total_epochs)
  File "/jet/home/xwang30/kl-nn/arch/train.py", line 272, in train
    self._save_checkpoint(epoch)
  File "/jet/home/xwang30/kl-nn/arch/train.py", line 245, in _save_checkpoint
    torch.save(ckp, PATH)
  File "/jet/home/xwang30/.conda/envs/kl-nn/lib/python3.10/site-packages/torch/serialization.py", line 966, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/jet/home/xwang30/.conda/envs/kl-nn/lib/python3.10/site-packages/torch/serialization.py", line 828, in _open_zipfile_writer
    return container(name_or_buffer)  # type: ignore[arg-type]
  File "/jet/home/xwang30/.conda/envs/kl-nn/lib/python3.10/site-packages/torch/